数据加载

In [1]:
import pandas as pd

# demo 验证
train_df = pd.read_csv('Data/Movie_RS.csv')
print(train_df.shape)
train_df.head(1)

(10000, 13)


,ID,Movie_Name,Movie_Score,Review_Count,Movie_Star_Distribution,Collect_Date,Username,Post_Date,Score,User_Comment,User_Comment_Distribution,Comment_Like_Count,Movie_Tags
0,0,"1988年的妮可 Nico, 1988",7.5,565,15.2%48.2%32.3%3.4%0.8%,2019-10-05,尾黑,2018-06-23,3,成本低廉的PPT电影，用Nico生命中最后一年发生的事给Nico的歌配上情节，倒不算尴尬。女...,66%31%3%,4,"['音乐', '电影', '儿子', '丝绒', '人物', '传记', '传记片', '歌..."


数据预处理。

In [2]:
# 只保留电影信息
train_df.drop_duplicates(
    subset=['Movie_Name'], keep='first', inplace=True)

# 打印电影的数据结构
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 9848
Data columns (total 13 columns):
ID                           26 non-null int64
Movie_Name                   26 non-null object
Movie_Score                  26 non-null float64
Review_Count                 26 non-null int64
Movie_Star_Distribution      26 non-null object
Collect_Date                 26 non-null object
Username                     26 non-null object
Post_Date                    26 non-null object
Score                        26 non-null int64
User_Comment                 26 non-null object
User_Comment_Distribution    26 non-null object
Comment_Like_Count           26 non-null int64
Movie_Tags                   26 non-null object
dtypes: float64(1), int64(4), object(8)
memory usage: 2.8+ KB


### 数据预处理

In [11]:
# 控制推荐质量，设置电影打分阈值 Threshold
Threshold = 7.5
train_df = train_df[train_df['Movie_Score'] > Threshold]

In [12]:
# 建立用户名和 id 映射的字典
user_dict = {value: index for index,
             value in enumerate(train_df['Username'].unique())}
# 开始对原数据进行转换
train_df['uid_int'] = train_df['Username'].apply(lambda x: user_dict[x])


# 建立电影名和 id 映射的字典
item_dict = {value: index for index, value in enumerate(
    train_df['Movie_Name'].unique())}
# 开始对原数据进行转换
train_df['item_int'] = train_df['Movie_Name'].apply(lambda x: item_dict[x])


# 用户 id 字典翻转
reverse_user_dict = {v: k for k, v in user_dict.items()}
# 电影 id 字典翻转
reverse_item_dict = {v: k for k, v in item_dict.items()}

设置电影和用户特征。

In [13]:
# 电影特征
items_f = ['Movie_Score', 'Review_Count', 'item_int', 'Movie_Name','Movie_Tags']

# 用户特征
users_f = ['uid_int','User_Comment_Distribution','Username']

### 基于标签的最近邻电影推荐

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# 定义近邻检索函数
def get_movie_recommendations(movie_realname, cosine_sim, topk):
    idx = item_dict[movie_realname]
    # 去除自身，自己和自己的计算结果是 1
    sim_scores = sorted(
        enumerate(cosine_sim[idx]), key=lambda x: x[1], reverse=True)[1:topk+1]
    return sim_scores

#### 标签向量化

In [15]:
# 初始化标签向量化函数,并设置最大的词数限制
content_vec = TfidfVectorizer(min_df=1)

# 文本向量化
count_matrix = content_vec.fit_transform(train_df['Movie_Tags'])

# 两两内容的相似度计算
%time cosine_sim_result = cosine_similarity(count_matrix)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 771 µs


假设当前电影 id，并打印详细信息。

In [16]:
# 输入电影名字
movie_realname = '一曲难忘 A Song to Remember'

# 打印电影信息
train_df.loc[train_df['Movie_Name'].isin([movie_realname])].drop_duplicates(
    'item_int', keep='first', inplace=False)[items_f].head()

,Movie_Score,Review_Count,item_int,Movie_Name,Movie_Tags
2973,7.8,2079,5,一曲难忘 A Song to Remember,"['电影', '音乐', '老师', '教授', '音乐课', '时候', '艺术', '有..."


通过建立的电影 id 向量，找到最近的 tokn 个电影。

In [17]:
topn = 5

# 开始进行进行近邻推荐,输出电影 id 和近邻计算的分数
%time ANN_List = get_movie_recommendations(movie_realname, cosine_sim_result, topn)
ANN_List

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 42.4 µs


[(6, 0.26741533732321265),
 (11, 0.23743498387740047),
 (9, 0.15544874263702108),
 (1, 0.14327359758576902),
 (7, 0.12276024698244593)]

打印近邻的电影信息。

In [18]:
# 数据去重，保留首先出现的数据。
ANN_List_Info = train_df.loc[train_df['item_int'].isin(
    [i[0] for i in ANN_List])].drop_duplicates('item_int', keep='first', inplace=False)

# 近邻检索的数据
ANN_List_Info[items_f].head()

,Movie_Score,Review_Count,item_int,Movie_Name,Movie_Tags
373,7.9,12803,1,42号传奇 42,"['成就', '励志', '桥段', '夫妻', '棒球', '电影', '核心', '数字..."
3679,7.9,4395,6,一轮明月,"['电影', '大师', '人物', '感觉', '人生', '主旋律', '流水账', '..."
4477,7.6,2620,7,七小福,"['电影', '故事', '时代', '师父', '传统', '感人', '片场', '京剧..."
5394,9.3,1010,9,三个臭皮匠 The Three Stooges,"['喜剧', '小时候', '时候', '经典', '电视', '电影', '电视剧', '..."
8811,8.0,3581,11,不朽真情 Immortal Beloved,"['音乐', '电影', '爱情', '故事', '有点', '剧情', '天才', '狗血..."


#### 应用推广
主要根据标签之间的相似度进行近邻检索进行推荐的一种方式。

1. 用户电影之间的电影近邻推荐。
2. 用户用户之间的用户近邻推荐。